In [ ]:
import torch
import torchvision
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [ ]:
model = torchvision.models.vgg16_bn(weights = torchvision.models.VGG16_BN_Weights.DEFAULT)

In [ ]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
model.classifier[-1] = torch.nn.Linear(in_features=4096 , out_features= 101)

In [ ]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=101, bias=True)
)

In [ ]:
from os.path import split
train_dataset = torchvision.datasets.Food101(root="./data",split = "train", download=True,
                                           transform=torchvision.transforms.Compose([
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Resize((227, 227)),
                                               torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                std=[0.229, 0.224, 0.225])
                                           ]))

test_dataset = torchvision.datasets.Food101(root="./data", split= "test", download=True,
                                           transform=torchvision.transforms.Compose([
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Resize((227, 227)),
                                               torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                std=[0.229, 0.224, 0.225])
                                           ]))


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"running on: {device}")

train_data_loader = DataLoader(dataset=train_dataset, batch_size=64)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=64)
criterion = torch.nn.CrossEntropyLoss()

epochs_num = 20
learning_rate = 1e-4
total_step = len(train_data_loader)
model = model.to(device=device)
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=learning_rate, weight_decay=0.005)
print(120 * '-')

running on: cuda
------------------------------------------------------------------------------------------------------------------------


In [ ]:

for epoch in range(20):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in tqdm(test_data_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

      print(f"Accuracy of the network on the {5000} validation images: {100*correct/total}%")


      print(120 * '-')
